In [12]:
import random
from Bio import SeqIO
from collections import defaultdict

def parse_fasta_by_gene(fasta_file):
    """
    Parses a FASTA file and groups sequences by gene.
    Assumes gene name is part of the sequence description.
    """
    gene_dict = defaultdict(list)
    for record in SeqIO.parse(fasta_file, "fasta"):
        # Adjust the parsing according to your file's format
        parts = record.description.split('|')  # Split the description by '|'
        gene_name = parts[2]  # Modify this index based on where the gene name is in your file
        gene_dict[gene_name].append(record)
    return gene_dict


def sample_genes(gene_dict, fraction=0.1):
    total_genes = len(gene_dict)
    num_to_sample = int(total_genes * fraction)
    sampled_genes = random.sample(gene_dict.keys(), num_to_sample)
    sampled_records = [record for gene in sampled_genes for record in gene_dict[gene]]
    return sampled_records, sampled_genes



In [13]:
# Load the FASTA file and group by genes
gene_sequences = parse_fasta_by_gene("/cellar/users/yumei/Documents/nlp_mRNA/P2_RNAseq_finetuning/P2_cortex_filtered_mouse_cdna_shuffled.fasta")

# Sample 10% of genes and their transcripts
sampled_sequences, sampled_gene_names = sample_genes(gene_sequences, fraction=0.1)

# Write sampled sequences to a new file
with open("/cellar/users/yumei/Documents/nlp_mRNA/P2_RNAseq_finetuning_by_gene/P2_cortex_finetuning_by_gene_test.fasta", "w") as output_handle:
    SeqIO.write(sampled_sequences, output_handle, "fasta")


/tmp/ipykernel_1142202/1552597487.py:22: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  sampled_genes = random.sample(gene_dict.keys(), num_to_sample)


In [14]:
# Filter out the sampled sequences from the original file and write the rest to another file
remaining_sequences = [record for gene, records in gene_sequences.items() if gene not in sampled_gene_names for record in records]

with open("/cellar/users/yumei/Documents/nlp_mRNA/P2_RNAseq_finetuning_by_gene/P2_cortex_finetuning_by_gene_nontest.fasta", "w") as output_handle:
    SeqIO.write(remaining_sequences, output_handle, "fasta")

In [16]:
print (sampled_gene_names[:5])

['Gm17081', 'Frmd6', 'Creb3l2', 'Lin7b', 'Zfp395']


In [17]:
#double checking:

from Bio import SeqIO

def extract_gene_names(fasta_file):
    gene_names = set()
    for record in SeqIO.parse(fasta_file, "fasta"):
        # Extract gene name from the record description
        # This needs to be adjusted based on how your file's headers are formatted
        # Example: if header is ">gene_name|other_info", use split('|')[0]
        gene_name = record.description.split('|')[2]  # Modify this as per your file's format
        gene_names.add(gene_name)
    return gene_names

# Path to your FASTA file
fasta_file = "/cellar/users/yumei/Documents/nlp_mRNA/P2_RNAseq_finetuning/P2_cortex_filtered_mouse_cdna_shuffled.fasta"

# Extract gene names
unique_gene_names = extract_gene_names(fasta_file)

# Print the number of unique gene names
print(f"Number of unique gene names: {len(unique_gene_names)}")


Number of unique gene names: 12902


In [18]:
# Path to your FASTA file
fasta_file = "/cellar/users/yumei/Documents/nlp_mRNA/P2_RNAseq_finetuning_by_gene/P2_cortex_finetuning_by_gene_test.fasta"

# Extract gene names
unique_gene_names = extract_gene_names(fasta_file)

# Print the number of unique gene names
print(f"Number of unique gene names: {len(unique_gene_names)}")


Number of unique gene names: 1290


In [19]:
# Path to your FASTA file
fasta_file = "/cellar/users/yumei/Documents/nlp_mRNA/P2_RNAseq_finetuning_by_gene/P2_cortex_finetuning_by_gene_nontest.fasta"

# Extract gene names
unique_gene_names = extract_gene_names(fasta_file)

# Print the number of unique gene names
print(f"Number of unique gene names: {len(unique_gene_names)}")


Number of unique gene names: 11612


In [21]:
from Bio import SeqIO

def is_gene_in_fasta(fasta_file, gene_name):
    for record in SeqIO.parse(fasta_file, "fasta"):
        # Extract gene name from the record description
        # Adjust the extraction method to match your file's format
        current_gene_name = record.description.split()[0]  # Modify as per your file's format
        if current_gene_name == gene_name:
            return True
    return False

# Path to your FASTA file
fasta_file = "/cellar/users/yumei/Documents/nlp_mRNA/P2_RNAseq_finetuning_by_gene/P2_cortex_finetuning_by_gene_nontest.fasta"
# Check if 'Frmd6' is in the file
gene_name = "Lin7b"
found = is_gene_in_fasta(fasta_file, gene_name)

if found:
    print(f"The gene {gene_name} is present in the file.")
else:
    print(f"The gene {gene_name} is not present in the file.")


The gene Lin7b is not present in the file.


In [22]:
#used mmseqs2 to remove similar sequences from P2_cortex_by_gene_nontest.fasta; the new file is P2_cortex_by_gene_filtered_nontest.fasta   

In [24]:
#shuffle the nontest file:import random
import random
from Bio import SeqIO

def shuffle_fasta(input_fasta, output_fasta, seed=42):
    # Set the random seed for reproducibility
    random.seed(seed)

    # Read sequences from the input file
    with open(input_fasta, 'r') as infile:
        sequences = list(SeqIO.parse(infile, 'fasta'))

    # Shuffle the sequence list
    random.shuffle(sequences)

    # Write the shuffled sequences to the output file
    with open(output_fasta, 'w') as outfile:
        SeqIO.write(sequences, outfile, 'fasta')

# Paths to your input and output files
input_fasta = "/cellar/users/yumei/Documents/nlp_mRNA/P2_RNAseq_finetuning_by_gene/P2_cortex_by_gene_filtered_nontest.fasta"
output_fasta = "/cellar/users/yumei/Documents/nlp_mRNA/P2_RNAseq_finetuning_by_gene/P2_cortex_finetuning_by_gene_nontest_shuffled.fasta"

# Shuffle sequences in the FASTA file
shuffle_fasta(input_fasta, output_fasta)




In [29]:
#count the original number of genes:

#double checking:

from Bio import SeqIO

def extract_gene_names(fasta_file):
    gene_names = set()
    for record in SeqIO.parse(fasta_file, "fasta"):
        # Extract gene name from the record description
        # This needs to be adjusted based on how your file's headers are formatted
        # Example: if header is ">gene_name|other_info", use split('|')[0]
        gene_name = record.description.split('|')[2]  # Modify this as per your file's format
        gene_names.add(gene_name)
    return gene_names

# Path to your FASTA file
fasta_file = "/cellar/users/yumei/Documents/nlp_mRNA/P2_RNAseq_finetuning/P2_cortex_filtered_mouse_cdna_shuffled.fasta"

# Extract gene names
unique_gene_names = extract_gene_names(fasta_file)

# Print the number of unique gene names
print(f"Number of unique gene names: {len(unique_gene_names)}")

# Calculate the number of gene names to sample (10% of the total)
num_to_sample = int(0.1 * len(unique_gene_names))

print (num_to_sample)

Number of unique gene names: 12902
1290


In [30]:
def sample_no_genes(gene_dict, num_to_sample):
    total_genes = len(gene_dict)
    sampled_genes = random.sample(gene_dict.keys(), num_to_sample)
    sampled_records = [record for gene in sampled_genes for record in gene_dict[gene]]
    return sampled_records, sampled_genes


In [34]:
# Load the FASTA file and group by genes
gene_sequences = parse_fasta_by_gene("/cellar/users/yumei/Documents/nlp_mRNA/P2_RNAseq_finetuning_by_gene/P2_cortex_finetuning_by_gene_nontest_shuffled.fasta")
print (len(gene_sequences))
print (num_to_sample)
# Sample 10% of genes and their transcripts
sampled_sequences, sampled_gene_names = sample_no_genes(gene_sequences, num_to_sample)

# Write sampled sequences to a new file
with open("/cellar/users/yumei/Documents/nlp_mRNA/P2_RNAseq_finetuning_by_gene/P2_cortex_finetuning_by_gene_validation.fasta", "w") as output_handle:
    SeqIO.write(sampled_sequences, output_handle, "fasta")


11588
1290


/tmp/ipykernel_1142202/3078029525.py:3: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  sampled_genes = random.sample(gene_dict.keys(), num_to_sample)


In [35]:
# Filter out the sampled sequences from the original file and write the rest to another file
remaining_sequences = [record for gene, records in gene_sequences.items() if gene not in sampled_gene_names for record in records]

with open("/cellar/users/yumei/Documents/nlp_mRNA/P2_RNAseq_finetuning_by_gene/P2_cortex_finetuning_by_gene_nonvalidation.fasta", "w") as output_handle:
    SeqIO.write(remaining_sequences, output_handle, "fasta")

In [36]:
from Bio import SeqIO

def extract_gene_names(fasta_file):
    gene_names = set()
    for record in SeqIO.parse(fasta_file, "fasta"):
        # Extract gene name from the record description
        # Adjust this based on your file's header format
        gene_name = record.description.split('|')[2]  # Modify this as per your file's format
        gene_names.add(gene_name)
    return gene_names

# Paths to your FASTA files
fasta_file1 = "P2_cortex_by_gene_filtered_nonvalidation.fasta"
fasta_file2 = "P2_cortex_finetuning_by_gene_validation.fasta"

# Extract gene names from both files
gene_names_file1 = extract_gene_names(fasta_file1)
gene_names_file2 = extract_gene_names(fasta_file2)

# Find common gene names
common_gene_names = gene_names_file1.intersection(gene_names_file2)

# Print the results
print(f"Number of common gene names: {len(common_gene_names)}")
if len(common_gene_names) > 0:
    print("Common gene names:")
    for name in common_gene_names:
        print(name)


Number of common gene names: 0


In [37]:
from Bio import SeqIO

def extract_gene_names(fasta_file):
    gene_names = set()
    for record in SeqIO.parse(fasta_file, "fasta"):
        # Extract gene name from the record description
        # Adjust this based on your file's header format
        gene_name = record.description.split('|')[2]  # Modify this as per your file's format
        gene_names.add(gene_name)
    return gene_names

# Paths to your FASTA files
fasta_file1 = "P2_cortex_by_gene_filtered_nonvalidation.fasta"
fasta_file2 = "P2_cortex_finetuning_by_gene_test.fasta"

# Extract gene names from both files
gene_names_file1 = extract_gene_names(fasta_file1)
gene_names_file2 = extract_gene_names(fasta_file2)

# Find common gene names
common_gene_names = gene_names_file1.intersection(gene_names_file2)

# Print the results
print(f"Number of common gene names: {len(common_gene_names)}")
if len(common_gene_names) > 0:
    print("Common gene names:")
    for name in common_gene_names:
        print(name)


Number of common gene names: 0
